# ALeRCE Xmatch service example

A short example for accessing the courrent data aviable on the ALeRCE servers for cross-match, using custom arbirtrary objects

In [1]:
!pip install alerce_xmatch

For documentation visit GitHub repository: https://github.com/alercebroker/xmatch_client

Or the PyPi project: https://pypi.org/project/alerce-xmatch/

Generate data example

In [2]:
import pandas as pd

In [3]:
#input
columns = ['oid', 'ra', 'dec']
values = [
    [0, 168.821096816667, 29.6921437083333],
    [1, 164.669160169091, 16.0180675727273],
    [2, 174.65028162381, 20.726187647619],
    [3, 154.4155987, 30.8199994785714]
]
df = pd.DataFrame(values, columns=columns)

## Using Catalog class

In [4]:
from alerce_xmatch import Catalog, TargetCatalog, OutputCols
import time

Create Catalog class with test data

In [5]:
my_catalog = Catalog(df=df, name="My catalog")

## Crossmatch with Gaia (a Catalog in DataBase)

start = time.time()

cross_catalog = my_catalog.crossmatch(TargetCatalog.GAIA)

end = time.time()

print("Catalog name: {}".format(cross_catalog.name))
print("Crossmatch take: {} s".format(end - start))
print("Result:")
cross_catalog.df

Catalog name: My catalog
Crossmatch take: 12.34043574333191 s
Result:


,My catalog_dec,My catalog_oid,My catalog_ra,GAIA-DR2_designation,GAIA-DR2_ra,GAIA-DR2_dec,GAIA-DR2_parallax,GAIA-DR2_pmra,GAIA-DR2_pmdec,GAIA-DR2_astrometric_pseudo_colour,...,GAIA-DR2_bp_rp,GAIA-DR2_bp_g,GAIA-DR2_g_rp,GAIA-DR2_radial_velocity,GAIA-DR2_teff_val,GAIA-DR2_lum_val,GAIA-DR2_classifier_name,GAIA-DR2_best_class_name,GAIA-DR2_best_class_score,separation
0,16.018068,1.0,164.66916,Gaia DR2 3981758871818873600,164.669179,16.018048,0.107938,-3.091704,-2.052673,1.652324,...,0.572648,0.185996,0.386652,NaN,6898.6665,NaN,nTransits:2+,RRAB,0.985119,0.093911


To much info? Let's do a crossmatch with a catalog in memory (it supposed to be faster) and just see our info

In [6]:
start = time.time()

cross_catalog = my_catalog.crossmatch(TargetCatalog.ASASSN, output=OutputCols.SOURCE)  # Just my info please

end = time.time()

print("Crossmatch take: {} s".format(end - start))
print("Result:")
cross_catalog.df

Crossmatch take: 0.9985611438751221 s
Result:


,separation,My catalog_oid,My catalog_ra,My catalog_dec
0,None,0.0,168.821097,29.692144
1,None,3.0,154.415599,30.819999
2,None,1.0,164.669160,16.018068
3,None,2.0,174.650282,20.726188


I know the ra, dec position, maybe that's info is not neccesary

In [7]:
cross_catalog = my_catalog.crossmatch(TargetCatalog.ZTF, output=OutputCols.TARGET, # Just yuor info
                                     radec=False) # Don't show me the position
cross_catalog.df

,separation,ZTF_index,ZTF_oid
0,None,1107927,ZTF19aadfkyz
1,None,6233821,ZTF19aadfkza
2,None,9711443,ZTF19aaknuwq
3,None,13445830,ZTF18aaxpvha
4,None,13448367,ZTF17aabtute
5,None,13448374,ZTF17aaajmde
6,None,13845503,ZTF18acbwbhp
7,None,14676185,ZTF18aaanyxu
8,None,15105850,ZTF18aabksxe


## Using just the DataFrame
If you don't want to use that annoying Catalog class, use the crossmatch static method instead

In [8]:
from alerce_xmatch import crossmatch

In [9]:
crossmatch(df, TargetCatalog.ZTF) # df was declared on cell 3

,ZTF_index,ZTF_oid,ZTF_ra,ZTF_dec,ra,dec,separation,input_catalog_oid,input_catalog_ra,input_catalog_dec
0,1107927,ZTF19aadfkyz,154.415950,30.819799,154.415774,30.819899,None,3.0,154.415599,30.819999
1,6233821,ZTF19aadfkza,154.416161,30.819826,154.415880,30.819913,None,3.0,154.415599,30.819999
2,9711443,ZTF19aaknuwq,164.669156,16.017554,164.669158,16.017811,None,1.0,164.669160,16.018068
3,13445830,ZTF18aaxpvha,164.669518,16.018026,164.669339,16.018047,None,1.0,164.669160,16.018068
4,13448367,ZTF17aabtute,164.669165,16.018068,164.669163,16.018068,None,1.0,164.669160,16.018068
5,13448374,ZTF17aaajmde,174.650284,20.726185,174.650283,20.726186,None,2.0,174.650282,20.726188
6,13845503,ZTF18acbwbhp,154.415609,30.819991,154.415604,30.819995,None,3.0,154.415599,30.819999
7,14676185,ZTF18aaanyxu,168.821000,29.692205,168.821048,29.692174,None,0.0,168.821097,29.692144
8,15105850,ZTF18aabksxe,168.821102,29.692145,168.821099,29.692144,None,0.0,168.821097,29.692144


You can also use the crossmatch method with a Catalog object, and the output will be a Catalog as well

In [10]:
crossmatch(my_catalog, TargetCatalog.LINEAR).df # Remember print just the DataFrame

,LINEAR_id,LINEAR_class,LINEAR_period,LINEAR_A,LINEAR_mmed,LINEAR_stdev,LINEAR_rms,LINEAR_Lchi2pdf,LINEAR_nObs,LINEAR_skew,...,LINEAR_gErr,LINEAR_rErr,LINEAR_iErr,LINEAR_zErr,ra,dec,separation,My catalog_oid,My catalog_ra,My catalog_dec
0,1863652,RR Lyr ab,0.635622,0.88,16.68,0.31,0.38,1.671,250,-0.29,...,0.01,0.01,0.01,0.01,174.650267,20.726207,None,2.0,174.650282,20.726188
1,23555046,RR Lyr ab,0.514367,0.77,15.62,0.25,0.27,2.268,303,-0.66,...,0.01,0.01,0.01,0.01,164.669183,16.018061,None,1.0,164.669160,16.018068


## An example with a bigger catalog

To check a bigger crossmatch, we're gonna use the _catalina.pkl_ dataframe

In [11]:
import pickle

with open("example_data/catalina.pkl", "rb") as file:
    catalina = pickle.load(file)

In [12]:
# Create catalog:
catalina_catalog = Catalog(df=catalina, name="Catalina Example")

result = catalina_catalog.crossmatch(TargetCatalog.ASASSN).df

print("It was found {} rows".format(len(result)))
print("Firsts rows:")
result.head()

It was found 34757 rows
Firsts rows:


,ASASSN_id,ASASSN_Other Names,ASASSN_LCID,ASASSN_ra,ASASSN_dec,ASASSN_Mean VMag,ASASSN_Amplitude,ASASSN_period,ASASSN_class,ASASSN_Url,...,ASASSN_Sllk Statistic,ASASSN_RF Regression Score,ASASSN_Classification Probability,ASASSN_Epoch (HJD),ra,dec,separation,Catalina Example_oid,Catalina Example_ra,Catalina Example_dec
0,ASASSN-V J204308.90-011533.7,CSS_J204308.9-011532,175106,310.78710,-1.25937,15.16,0.38,0.4294613,EB,/database/light_curves/283889,...,0.28,0.86,0.998,2.457904e+06,310.787175,-1.259241,None,CSS_J204308.9-011532,310.787250,-1.259111
1,ASASSN-V J013051.79+313851.1,AM Psc,51150,22.71578,31.64752,13.83,0.80,107.8898995,SR,/database/light_curves/381090,...,0.60,0.70,1.000,2.457588e+06,22.715744,31.647552,None,CSS_J013051.7+313851,22.715708,31.647583
2,ASASSN-V J234254.97+384328.2,CSS_J234255.0+384328,131254,355.72903,38.72450,14.83,0.93,216.351107,SR,/database/light_curves/382465,...,0.73,0.66,1.000,2.457309e+06,355.729223,38.724486,None,CSS_J234255.0+384328,355.729417,38.724472
3,ASASSN-V J165903.31+433053.6,V0782 Her,56887,254.76379,43.51490,14.19,1.70,228.2831729,SR,/database/light_curves/382544,...,0.26,0.87,1.000,2.458019e+06,254.763812,43.514881,None,CSS_J165903.3+433053,254.763833,43.514861
4,ASASSN-V J055246.35-024207.0,CSS_J055246.3-024207,424610,88.19312,-2.70194,15.55,0.46,2.1161883,EA,/database/light_curves/286677,...,0.32,0.84,0.998,2.457335e+06,88.193122,-2.701942,None,CSS_J055246.3-024207,88.193125,-2.701944


In [13]:
# Crossmatch with ZTF
result = catalina_catalog.crossmatch(TargetCatalog.ZTF).df

print("It was found {} rows".format(len(result)))
print("Firsts rows:")
result.head()

It was found 45052 rows
Firsts rows:


,ZTF_index,ZTF_oid,ZTF_ra,ZTF_dec,ra,dec,separation,Catalina Example_oid,Catalina Example_ra,Catalina Example_dec
0,1871,ZTF18accduwj,131.984491,27.225956,131.984495,27.225950,None,CSS_J084756.2+271333,131.984500,27.225944
1,6148,ZTF18actbgye,82.033952,-6.872630,82.033872,-6.872579,None,CSS_J052808.1-065221,82.033792,-6.872528
2,9461,ZTF19aamnvqt,121.353403,6.106200,121.353452,6.105989,None,CSS_J080524.8+060620,121.353500,6.105778
3,10020,ZTF19aapcpqx,106.680897,40.673811,106.681094,40.673989,None,CSS_J070643.5+404027,106.681292,40.674167
4,11245,ZTF17aaahmvb,97.973156,49.054054,97.973057,49.054207,None,CSS_J063153.5+490315,97.972958,49.054361


In [14]:
with open("../example_data/agn_catalog.pickle", "rb") as file:
    big_df = pickle.load(file)

start = time.time()
result = crossmatch(big_df, TargetCatalog.TNS)
end = time.time()

print("It takes {} s".format(end - start))
print("It was found {} rows".format(len(result)))
print("Firsts rows:")
result.head()

It takes 39.94607448577881 s
It was found 761 rows
Firsts rows:


,TNS_id,TNS_RA_orig,TNS_DEC_orig,TNS_class,TNS_Redshift,TNS_Host Name,TNS_Host Redshift,TNS_Discovering Group/s,TNS_Classifying Group/s,TNS_Associated Group/s,...,TNS_ra,TNS_dec,TNS_aitoff_x,TNS_aitoff_y,ra,dec,separation,input_catalog_name,input_catalog_ra,input_catalog_dec
0,AT 2018kxt,04:36:52.120,-52:16:38.64,Unknown,NaN,NaN,NaN,GaiaAlerts,NaN,GaiaAlerts,...,69.217167,-52.277400,-78.085702,-61.325931,69.217172,-52.277402,None,WISEA J043652.12-521638.6,69.217177,-52.277404
1,AT 2018kvz,07:44:20.390,-69:19:07.10,Unknown,NaN,NaN,NaN,GaiaAlerts,NaN,GaiaAlerts,...,116.084958,-69.318639,-29.514722,-73.858661,116.084972,-69.318648,None,WISEA J074420.39-691907.1,116.084986,-69.318657
2,AT 2018kvr,05:33:02.520,-48:25:21.47,Unknown,NaN,NaN,NaN,GaiaAlerts,NaN,GaiaAlerts,...,83.260500,-48.422631,-74.382066,-56.087700,83.260424,-48.422669,None,J053302.48-482521.7,83.260348,-48.422708
3,AT 2018kuv,05:38:50.360,-44:05:08.95,Unknown,NaN,NaN,NaN,GaiaAlerts,NaN,GaiaAlerts,...,84.709833,-44.085819,-78.434362,-51.401910,84.709836,-44.085818,None,PKS 0537-441,84.709840,-44.085816
4,AT 2019nr,23:58:02.130,-45:55:18.80,Unknown,NaN,NaN,NaN,GaiaAlerts,NaN,GaiaAlerts,...,359.508875,-45.921889,125.027521,-64.559123,359.508896,-45.921897,None,PKS 2355-461,359.508916,-45.921905
